# Testing Data Lake Access using Service Account

In [13]:
from google.cloud import bigquery_datatransfer

# The project where the query job runs is the same as the project
# containing the destination dataset.
env = "exp"

project_id = f"lt-dia-analytics-{env}-cfs"
dataset_id = "ppl_analytics_output_cfs"

table_id = "loyalty_identity_curr"

# This service account will be used to execute the scheduled queries. Omit
# this request parameter to run the query as the user with the credentials
# associated with this client.
service_account_name = f"sa-lt-dia-{env}-cfs-lylty@lt-dia-analytics-{env}-cfs.iam.gserviceaccount.com"

# Use standard SQL syntax for the query.
query_string = f"""
CREATE OR REPLACE TABLE {project_id}.{dataset_id}.{table_id}
AS (SELECT * FROM lt-dia-lake-{env}-consume.entity_b2c.{table_id} LIMIT 10);
"""

transfer_client = bigquery_datatransfer.DataTransferServiceClient()
parent = transfer_client.common_project_path(project_id)

transfer_config = bigquery_datatransfer.TransferConfig(
    destination_dataset_id=dataset_id,
    display_name=f"Copying sample {table_id}",
    data_source_id="scheduled_query",
    params={
        "query": query_string,
        # "destination_table_name_template": f"{table_id}",
        # "write_disposition": "WRITE_TRUNCATE",
        # "partitioning_field": "busn_eff_dtm",
    },
    schedule="every 24 hours",
)

transfer_config = transfer_client.create_transfer_config(
    bigquery_datatransfer.CreateTransferConfigRequest(
        parent=parent,
        transfer_config=transfer_config,
        service_account_name=service_account_name,
    )
)

print(f"Created {table_id} scheduled query '{transfer_config.name}'")
    
req_tables = ['loyalty_lcl_transaction_summary',
              'loyalty_sdm_transaction_summary',
              'loyalty_lcl_transaction_scancode',
              'loyalty_sdm_transaction_scancode']

for table_id in req_tables:

    # This service account will be used to execute the scheduled queries. Omit
    # this request parameter to run the query as the user with the credentials
    # associated with this client.
    service_account_name = f"sa-lt-dia-{env}-cfs-lylty@lt-dia-analytics-{env}-cfs.iam.gserviceaccount.com"

    # Use standard SQL syntax for the query.
    query_string = f"""
    CREATE OR REPLACE TABLE {project_id}.{dataset_id}.{table_id}
    AS (SELECT * FROM lt-dia-lake-{env}-consume.transactional_loyalty.{table_id}
        WHERE trans_dt = "2023-09-15" LIMIT 10);
    """
    
    transfer_client = bigquery_datatransfer.DataTransferServiceClient()
    parent = transfer_client.common_project_path(project_id)

    transfer_config = bigquery_datatransfer.TransferConfig(
        destination_dataset_id=dataset_id,
        display_name=f"Copying sample {table_id}",
        data_source_id="scheduled_query",
        params={
            "query": query_string,
            # "destination_table_name_template": f"{table_id}",
            # "write_disposition": "WRITE_TRUNCATE",
            # "partitioning_field": "busn_eff_dtm",
        },
        schedule="every 24 hours",
    )

    transfer_config = transfer_client.create_transfer_config(
        bigquery_datatransfer.CreateTransferConfigRequest(
            parent=parent,
            transfer_config=transfer_config,
            service_account_name=service_account_name,
        )
    )

    print(f"Created {table_id} scheduled query '{transfer_config.name}'")

Created loyalty_identity_curr scheduled query 'projects/653530402634/locations/northamerica-northeast1/transferConfigs/6547d760-0000-2a22-85e6-14223bbf42a6'
Created loyalty_lcl_transaction_summary scheduled query 'projects/653530402634/locations/northamerica-northeast1/transferConfigs/6548ab8a-0000-2f94-83c1-089e08268228'
Created loyalty_sdm_transaction_summary scheduled query 'projects/653530402634/locations/northamerica-northeast1/transferConfigs/6547e100-0000-2ebb-aa9d-3c286d3c295e'
Created loyalty_lcl_transaction_scancode scheduled query 'projects/653530402634/locations/northamerica-northeast1/transferConfigs/6547bade-0000-254c-b878-14223bb5164a'
Created loyalty_sdm_transaction_scancode scheduled query 'projects/653530402634/locations/northamerica-northeast1/transferConfigs/6548469c-0000-2ee2-8231-089e0828dd08'
